# Plotting Power ROIs on an interactive glass brain using `nilearn`

Example output:

<img src="glassBrain_example.png">

In [60]:
import numpy as np
from nilearn import plotting
from matplotlib import cm

def generate_cmap_categorical(nets):
    colors = [0]*nets.shape[0]
    n_subnets = np.unique(nets).size
    # most cmaps have 256 values, and we want to grab colors evenly spaced throughout the spectrum
    # if n_subnets > 10, you'll want to use a diverging color map and the scaling_factor
    scaling_factor = int(np.floor(256/(n_subnets+1)))
    for i in range(1,n_subnets+1):
        idxs = np.where(nets == i)[0]
        for x in idxs: 
            # you can change the color palette below (i.e. cm.Spectral(i))
            # if using a diverging color palette remember to do i*scaling_factor:
            colors[int(x)] = cm.hsv(i*scaling_factor)
            
            # if n_subnets < 10 you can use a categorical cmap and no scaling_factor:
            #colors[int(x)] = cm.Set1(i)
    return colors

# if you only care about a subset of networks 
def generate_cmap_categorical_selectNets(nets, selected_nets):
    colors = [0]*nets.shape[0]
    n_subnets = np.unique(nets).size
    # most cmaps have 256 values, and we want to grab colors evenly spaced throughout the spectrum
    # if n_subnets > 10, you'll want to use a diverging color map and the scaling_factor
    scaling_factor = int(np.floor(256/(n_subnets+1)))
    for i in range(n_subnets):
        idxs = np.where(nets == selected_nets[i])[0]
        for x in idxs: 
            # you can change the color palette below (i.e. cm.Spectral(i))
            # if using a diverging color palette remember to do i*scaling_factor
            #colors[int(x)] = cm.Spectral(i*scaling_factor)
            
            # if n_subnets < 10 you can use a categorical cmap and no scaling_factor
            colors[int(x)] = cm.Dark2(i)
    return colors

def generate_cmap_continuous(scores):
    colors = [0]*scores.shape[0]
    # scale the values in scores to range(256) and grab color from cmap
    for i in range(scores.shape[0]):
        scaled = (scores[i] - np.min(scores))/(np.max(scores) - np.min(scores))*255 
        colors[i] = cm.BuPu(int(np.round(scaled)))
    return colors

Basic running:

In [57]:
# load in data
data = np.loadtxt(open('PowerParcellation_coordinates_networklabels.csv','rb'), 
                  delimiter=",",usecols = (1,2,3), skiprows=1)
networks = np.loadtxt(open('PowerParcellation_coordinates_networklabels.csv','rb'), 
                  delimiter=",",usecols = (4), skiprows=1)

# mapping network numbers to labels
network_labels = {1:"Somatomotor - hand", 2: "Somatomotor - faces",
                 3:"Cingulo-opercular", 4:"Auditory", 5:"Default Mode",
                 6:"Memory Retrieval", 7:"Visual", 8:"Fronto-parietal",
                 9:"Salience", 10:"Subcortical", 11:"Ventral Attention",
                 12:"Dorsal Attention", 13:"Cerebellum"}

# removing ROIs that are not associated with known subnetworks
dat = data[np.where(networks > 0)]
nets = networks[np.where(networks > 0)]

colors = generate_cmap_categorical(nets)

# generate plot and view in new browser window
# for best results in saving - take a screenshot of the image in browser
view = plotting.view_markers(data, colors, marker_size=8) 
view.open_in_browser()

If you only care about a few subnetworks:

In [50]:
# if you only care about a few subnetworks you can use the code here:
selected_nets = [3,5,6,8,9,11,12]

dat = data[np.where(np.isin(networks, selected_nets))]
nets = networks[np.where(np.isin(networks, selected_nets))]

colors = generate_cmap_categorical_selectNets(nets,selected_nets)

# generate plot and view in new browser window
# for best results in saving - take a screenshot of the image in browser
view = plotting.view_markers(dat, colors, marker_size=8) 
view.open_in_browser()

If you want to color using a continuous score:

In [72]:
scores = np.random.uniform(0,1,data.shape[0])
colors = generate_cmap_continuous(scores)

# generate plot and view in new browser window
# for best results in saving - take a screenshot of the image in browser
view = plotting.view_markers(data, colors, marker_size=8) 
view.open_in_browser()

I used the code below to generate the images corresponding to the first line of Yujun's table: WM task within-network. To get corresponding images for the other lines in the table, change `selected_nets` to have the correct network ids, which can be identified from the `network_labels` dictionary above. 

In [73]:
# if you only care about a few subnetworks you can use the code here:
selected_nets = [5,8,9]

networks = np.loadtxt(open('PowerParcellation_coordinates_networklabels.csv','rb'), 
                  delimiter=",",usecols = (4), skiprows=1)

dat = data[np.where(networks > 0)]
nets = networks[np.where(networks > 0)]

nets[np.where(np.isin(nets, selected_nets, invert=True))] = 0
for i in range(len(selected_nets)):
    nets[np.where(nets == selected_nets[i])] = i+1

colors = generate_cmap_continuous(nets)

# generate plot and view in new browser window
# for best results in saving - take a screenshot of the image in browser
view = plotting.view_markers(dat, colors, marker_size=10) 
view.open_in_browser()

